<a href="https://colab.research.google.com/github/svshift1/urban_python_svshift/blob/main/module_15/module_15_2_%D0%98%D1%81%D1%82%D0%BE%D1%80%D0%B8%D1%8F_%D0%B8_%D0%A0%D0%B0%D0%B7%D0%B2%D0%B8%D1%82%D0%B8%D0%B5_%D0%98%D0%98_%D0%94_%D0%97_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Первая цифровая и аналоговая реализация математической модели нейрона - перцептрон Розенблата.

Реализуем операцию 2-И-не -- реализуемую аппаратно как двухэмиттерный транзистор на базе которого можно реализовать всё остальное.

In [1]:
import numpy as np

# Функция активации (шаговая функция)
def step_function(x):
    return np.where(x >= 0, 1, 0)

class Perceptron:
    def __init__(self, input_size, learning_rate=0.01, epochs=1000):
        self.W = np.zeros(input_size + 1)
        self.learning_rate = learning_rate
        self.epochs = epochs

    def predict(self, x):
        return step_function(np.dot(self.W, x))

    def train(self, X, y):
        for _ in range(self.epochs):
            for xi, target in zip(X, y):
                xi = np.insert(xi, 0, 1)  # Вставка смещения (bias)
                prediction = self.predict(xi)
                self.W += self.learning_rate * (target - prediction) * xi

# Данные для обучения (И, ИЛИ)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([1, 1, 1, 0])  # Операция И-Не

perceptron = Perceptron(input_size=2)
perceptron.train(X, y)

# Тестирование
for xi in X:
    xi_with_bias = np.insert(xi, 0, 1)  # Вставка смещения (bias) для тестирования
    print(f"{xi} -> {perceptron.predict(xi_with_bias)}")

[0 0] -> 1
[0 1] -> 1
[1 0] -> 1
[1 1] -> 0


попробуем импликацию:


In [2]:
# Данные для обучения (И, ИЛИ)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([1, 1, 0, 1])  # Операция X --> Y

perceptron = Perceptron(input_size=2)
perceptron.train(X, y)

# Тестирование
for xi in X:
    xi_with_bias = np.insert(xi, 0, 1)  # Вставка смещения (bias) для тестирования
    print(f"{xi} -> {perceptron.predict(xi_with_bias)}")

[0 0] -> 1
[0 1] -> 1
[1 0] -> 0
[1 1] -> 1


А теперь XOR --
читал, что ее реально реализовать одним перцептроном с двумя входами+биасом и хитрой активаторной функцией - полиномиальная (просто к весам добавляются коэффициенты этого полинома) + сигмоида -- такой перцептрон будет делить плосколсть на параллельные полосы

In [3]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])  # Операция X ^ Y

for e in [1000, 10000, 20000, 50000] :
  perceptron = Perceptron(input_size=2, epochs=e)
  perceptron.train(X, y)

  print(f'----------------\nepochs: {e}')
  # Тестирование
  for xi in X:
      xi_with_bias = np.insert(xi, 0, 1)  # Вставка смещения (bias) для тестирования
      print(f"   {xi} -> {perceptron.predict(xi_with_bias)}")

----------------
epochs: 1000
   [0 0] -> 1
   [0 1] -> 1
   [1 0] -> 0
   [1 1] -> 0
----------------
epochs: 10000
   [0 0] -> 1
   [0 1] -> 1
   [1 0] -> 0
   [1 1] -> 0
----------------
epochs: 20000
   [0 0] -> 1
   [0 1] -> 1
   [1 0] -> 0
   [1 1] -> 0
----------------
epochs: 50000
   [0 0] -> 1
   [0 1] -> 1
   [1 0] -> 0
   [1 1] -> 0


на выходе получилось тупо  not X[0]

если учесть, что   x^y == (x|y)  &  !(x&y)   то понятно откуда берется оценка 5 слоев в mlp классификаторе. при этом 5000 итераций ему было мало
пробовал при 4 слоях -- не всегда сходилась успешно даже на 50000 итерациях

In [19]:
from sklearn.neural_network import MLPClassifier
import numpy as np

# Данные для обучения (XOR)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])  # Операция XOR

# Создание и обучение MLP-классификатора
mlp = MLPClassifier(hidden_layer_sizes=(4,), activation='relu', max_iter=50000, solver='adam')
mlp.fit(X, y)

# Тестирование
for xi in X:
    print(f"{xi} -> {mlp.predict([xi])[0]}")

[0 0] -> 0
[0 1] -> 1
[1 0] -> 1
[1 1] -> 0
